# Big data - Весна 2024 "Сферический Велопрокат в Вакууме"

## Задание

>Учащимся выдается датасет с многолетними накопленными историческими
>данными о поездках велопроката Divvy Bikes. На основании этого датасета
>нужно оценить объем рынка и характер его динамики, а затем разработать
>модель юнит-экономики поездки для различных сценариев использования
>сервиса: разовая покупка, месячная подписка и т.д.
>Тезисный план:
>1. Разведочный анализ датасета
>2. Подготовка аналитических витрин данных
>3. Расширенный анализ витрин с целью оценки объема рынка, характера
>динамики и прогнозирования роста
>4. Разработка модели юнит-экономики поездки
>5. Анализ чувствительности по основным параметрам модели
>6. Визуализация ключевых показателей
>7. Подготовка рассказа, презентации и дашборда с основными
результатами и выводами

## Импорт Библиотек и загрузка датасетов

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr, spearmanr, chi2_contingency, ttest_ind, mannwhitneyu, shapiro, permutation_test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import re

pd.options.display.max_columns = None
pd.options.display.max_rows = None
%matplotlib inline

C:\Users\gosha\AppData\Local\Temp\ipykernel_10748\3043270857.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
x = 2020
df = pd.read_csv(f"{x}-cleared-divvy-tripdata.csv", sep=",")

## Предобработка данных

In [ ]:
def check_data(data_df: pd.DataFrame):

    """
        Функция для просмотра сводной информации о датасете
    """
    print ('\033[1m' + 'Сводная информация об исходных данных'+ '\033[0m')
    print(data_df.info())
    print(data_df.shape)
        
    missed_cells = data_df.isnull().sum().sum()/(data_df.shape[0]*(data_df.shape[1]-1)) # пропущенные ячейки
    missed_rows = sum(data_df.isnull().sum(axis = 1)>0)/data_df.shape[0]
    print ('\033[1m' + '\nПроверка пропусков'+ '\033[0m')
    print ('Количество пропусков: {:.0f}'.format(data_df.isnull().sum().sum()))
    print ('Доля пропусков: {:.1%}'.format(missed_cells)+ '\033[0m')
    print ('Доля строк, содержащих пропуски: {:.1%}'.format(missed_rows))

    ## Проверим дубликаты
    print ('\033[1m' + '\nПроверка на дубликаты'+ '\033[0m')
    print('Количество полных дубликатов: ', data_df.duplicated().sum())
        
    ## Посмотрим на сами данные
    print ('\033[1m' + '\nПервые десять строк датасета'+ '\033[0m')
    display(data_df.head(10))
    
    print('\033[1m' + '\nОписание количественных данных:'+ '\033[0m')
    display(data_df.describe().T)
    
    print('\033[1m' + '\nОписание категориальных данных:'+ '\033[0m')
    display(data_df.describe(include='object').T) 
    
    
    print('\033[1m' + '\nВывод уникальных значений по каждому категориальному признаку без учета id:'+ '\033[0m')    
    df_object = data_df.select_dtypes(include='object').columns
    
    for i in df_object:
        if data_df[i].nunique() > 10:
            continue
        print('\033[1m' + ('_' * 10) + '\033[0m')
        display(data_df[i].value_counts())


In [ ]:
def plot_hist(data, col_column):
    '''
    Функция отрисовки гистограмм и ящика с усами для количественных переменных.
    На вход: исходная таблица и список количественных переменных.
    На выходе: графики
    '''
    rows = len(col_column)
    f, ax = plt.subplots(rows, 2, figsize=(8, 16))
    f.tight_layout()
    f.set_figheight(15)
    f.set_figwidth(8)
    plt.rcParams.update({'font.size': 12})
    
    for i, col in enumerate(col_column):         
        sns.histplot(data[col], kde=True, bins=16, ax = ax[i, 0])                    
        sns.boxplot(data[col], ax = ax[i, 1])

        ax[i, 0].set_xlabel(col)
        ax[i, 1].set_xlabel(col)
        ax[i, 0].set_ylabel('Количество')
        ax[i, 1].set_ylabel("")
    plt.suptitle("Гистограмма и ящик с усами для количественных данных", fontsize=22, y=1.01)
    plt.show()

In [ ]:
# plot_hist(df, ["start_lat", "start_lng", "end_lat", "end_lng"])

In [ ]:
check_data(df)

**Категориальные столбцы с фиксированными возможными значениями проверены** в функции и дальнейшей обработки не потребуют

Важное замечание: мы не будем отбрасывать выбросы с помощью с помощью перцентилей в столбце со временем поездки так как столбец имеет экспоненциалльное распределение, но и не учитывать его (то-есть перейти к более большим величинам мы не можем потому что большее число клиентов пользуются велосипедами на короткий промежуток времени (меньше часа)) мы не можем

In [ ]:
df.isna().sum()

### ride_id

Уникальный идентификатор каждой поездки

In [ ]:
df["ride_id"].duplicated().sum()

### started_at и ended_at

**started_at** : дата и время начала поездки  
**ended_at** : дата и время окончания поездки

In [ ]:
# Исправляю формат данных столбцов started_at и ended_at на дату
df["started_at"] = pd.to_datetime(df["started_at"])
df["ended_at"] = pd.to_datetime(df["ended_at"])
# df.info()

In [ ]:
df["ride_duration"] = (df["ended_at"] - df["started_at"]) # столбец содержащий время поездки

In [ ]:
# df = df[df["ride_duration"] < pd.Timedelta(days=1)]
# sns.histplot(df["ride_duration"].apply(lambda x: x.total_seconds()/3600))

В датасете имеются ошибки с датой начала и конца поездки, а также хоть и на сайте написано что данные предобрабатываются от случайного срабатывания (человек пытался переустановить велосипед на стоянку чтобы убедиться в его безопасности или это может быть ложный старт) по факту в датасете их полным полно

In [ ]:
print(df.shape[0])
bigger = df[df["started_at"] > df["ended_at"]]
lower_1m = df[df["ride_duration"] < pd.Timedelta(minutes=1)]
df = df[df["started_at"] < df["ended_at"]]
df["fix"] = df["end_station_name"].apply(lambda x: x in ("HUBBARD ST BIKE CHECKING (LBS-WH-TEST)", 
                                                         "Base - 2132 W Hubbard Warehouse",
                                                         "hubbard_test_lws", 
                                                         "WATSON TESTING - DIVVY")) + \
            df["start_station_name"].apply(lambda x: x in ("HUBBARD ST BIKE CHECKING (LBS-WH-TEST)", 
                                                           "Base - 2132 W Hubbard Warehouse",
                                                           "hubbard_test_lws", 
                                                           "WATSON TESTING - DIVVY"))

fix_df = df[df["fix"]]
print(fix_df.shape)
df = df[df["fix"] == False]

day = df[df["ride_duration"] > pd.Timedelta(days=1)] # больше часа на велосипеде катаются всего 5000 записей из 300 тысяч
print(day.shape)
print(df.sort_values(by="ride_duration", ascending=False).shape)
print(df.sort_values(by="ride_duration").iloc[-1])

df = df[df["ride_duration"] > pd.Timedelta(minutes=1)]
df = df[df["ride_duration"] < pd.Timedelta(days=1)]
print(df.shape[0])
print(df["ride_duration"].describe())

Мы удаляем значения с длительностью поездки больше дня так как таких значений всего около 2500 тысяч и нам не так важны эти кейсы при анализе рынка  
Хотя можно попробовать проаанализировать эти данные отедльно и сказать что то по ним

Максимум длительности поездки 108 дней  
Точка конца поездки подозрительно близко находится с не очень людным местом около берега  
Могу предположить что велосипед бросили где то около моря после чего его кто то нашел и поставил на станцию через 4 месяца  

Заметил подозрительные значения HUBBARD ST BIKE CHECKING (LBS-WH-TEST) и узнал что это рементные станции которые divvy использует для ремонта велосипедов  
Мы должны удалить из датасета значения с точками старта и конца в следующих точках:
- Base - 2132 W Hubbard Warehouse  
- HUBBARD ST BIKE CHECKING (LBS-WH-TEST)  
- hubbard_test_lws  
- WATSON TESTING - DIVVY

**Поездок совершенных сотрудниками для осуществления ремонта оказалось 509**
Схораним это в отдельный датасет чтобы использовать потом

In [ ]:
sns.histplot(df["ride_duration"].apply(lambda x: x.total_seconds()/3600))

In [ ]:
df["day"] = df["started_at"].apply(lambda x: x.day)
df["month"] = df["started_at"].apply(lambda x: x.month)
df["year"] = df["started_at"].apply(lambda x: x.year)

In [ ]:
df["month"].unique().size > 1

In [ ]:
df["year"].unique().size > 1

### Названия и id точки начала поездки и конца

- start_station_name: название станции, с которой началось путешествие
- start_station_id: уникальный идентификатор станции, с которой началась поездка
- end_station_name: название станции, на которой закончилась поездка
- end_station_id: уникальный идентификатор станции, на которой закончилась поездка

In [ ]:
data = df[["start_station_name", "start_station_id", "end_station_name", "end_station_id"]]
print(data.isna().sum())
missed_cells = data.isna().sum().sum()/(data.shape[0]*(data.shape[1]))
print ('Доля пропусков: {:.1%}'.format(missed_cells)+ '\033[0m')

При исследовании данных столбцов было замечено **очень много пропусков** (15% от объема данных)

Посторили график и обнаружили зависимость между отсутствующими значениями и типом арендуемого велосипеда

In [ ]:
max_day = sorted(df["day"].unique())[-1] # переменная содержащая цифру по счету максимального дня в месяце

# print(df[["start_station_name", "start_lat", "start_lng", "end_lat", "end_lng"]].groupby(["start_station_name"]).head(2))

fig, ax = plt.subplots(2, 2, figsize=(12, 7))

sns.histplot(df["day"], kde=True, bins=max_day, ax=ax[0, 0])
ax[0, 0].set_ylabel("Аренда велосипедов по дням")
sns.histplot(df["rideable_type"], ax=ax[1, 0])
ax[1, 0].set_ylabel("Доля электро и обычных велосипедов")
ax[0, 0].set_title("До", fontsize=14)

data = df.dropna()
print(data.isna().sum())
print(data.shape[0])

sns.histplot(data["day"], kde=True, bins=max_day, ax=ax[0, 1])
sns.histplot(data["rideable_type"], ax=ax[1, 1])
ax[1, 1].set_ylabel("")
ax[0, 1].set_ylabel("")
ax[0, 1].set_title("После", fontsize=14)
plt.suptitle("Сравнение данных до\nи после удаления", fontsize=16)
# plt.savefig("Result_of_analisys_nan")

По графикам видно что ошибки в данных связаны с арендой электровелосипедов и если удалить эти данные мы потеряем львиную долю рынка электровелосепедов и в следствии репрезентативность выборки  
**Данные нужно восстанавливать**  
Либо удалять равную долю данных по обычным велосипедам

Если посмотреть на распределение долей между обычными и электро- велосипедами то можно заметить сильный скос (после удаления данных в которых есть пробелы) в сторону обычных велосипедов, а это значит что большинство записей с пропусками это электровелосипеды

Допустим

**А почему так происходит?**  
Есть довольно простой ответ  

Если посмотреть по карте то можно заметить что очень часто велосипеды оставляют просто вне стоянки где попало

![Отдельно стоящие велосипеды](./Отдельные_велосипеды.png)

Зеленое - Стоянки для велосипедов  
Красное - электро-велосипеды (ebike)  
**Что с этим делать?**  
Один из простых вариантов **просто заполнить пропуски меткой что данные велосипеды взяты и припаркованы вне стоянки**

In [ ]:
df.fillna({"start_station_name": "droped", 
           "end_station_name": "droped",
           "start_station_id": "droped",
           "end_station_id": "droped"}, inplace=True)

df[["start_station_name", "start_station_id", "end_station_name", "end_station_id"]].isna().sum()

### Координаты начала и конца поездки

- start_lat: широта начальной станции
- start_lng: долгота начальной станции
- end_lat: широта конечной станции
- end_lng: долгота конечной станции

**Задание: разобраться с пропусками в координатах конца поездки**  
У нас есть данные с временем аренды больше дня у которых отсутствует точка конца поездки  
Это ошибка в данных, но как это можно объяснить?

In [ ]:
na_end_coord = df[df["end_lat"].isna() == True]
print(na_end_coord.sort_values(by="ride_duration", ascending=False).head())

In [ ]:
# sns.bar(data["rideable_type"])

In [ ]:
sns.histplot(na_end_coord["ride_duration"].apply(lambda x: x.total_seconds()/3600))
plt.show()

In [ ]:
# sns.histplot(df["ride_duration"].apply(lambda x: x.total_seconds()/3600))
# plt.show()
# print(df.sort_values(by="ride_duration", ascending=False).head())

Вероятнее всего это что-то одно либо все сразу:
- поездки сотрудников (хотя указано что данные от этого предобработаны)
- украденные велосипеды которые куда нибудь укатили и разобрали на запчасти 
- с велосипедами произошел несчастный случай (велосипед утопили, человека на велосипеде сбили и так далее)

**Это мы отфильтровываем и в конечный датасет не берем**

In [ ]:
df.dropna(inplace=True)
# df.isna().sum()

In [ ]:
# sns.histplot(df["ride_duration"].apply(lambda x: x.total_seconds()/3600))
# plt.show()
print(df.sort_values(by="ride_duration", ascending=False).head())

In [ ]:
# df[df["ride_duration"] > pd.Timedelta(hours=10)].head(20)

Получшившееся в результате очистки максимальное значение времени поездки очень даже согласовывается с его данными так как это может быть самый обыкновыенный пользователь который взял велосипед покататься на день возле дома, покатался, где то переночевал и вернулся обратно домой

### Обогащение данных

In [ ]:
# stations = df.groupby(["start_station_name"])[["start_station_id", "start_lat", "start_lng"]].head(5).sort_values(by="start_station_id")
# stations

In [ ]:
df["day_of"] = df["started_at"].apply(lambda x: x.day_name())
# df["day_of"].head()
df["is_hol"] = df["started_at"].apply(lambda x: x.day_of_week in (5, 6))
# df[["day_of", "is_hol"]][df["is_hol"]].head(40)

**Вопрос: У нас есть равные и больше даты старта и конца   
Это все ошибки произошедшие по тем или иным причинам  
Стоит ли это удалять  
Это может быть не ошибка в данных, а может быть пользователь просто взял и сразу поставил велосипед**  
**UPD**: Это ошибка в нормальной работе системы а значит скорее всего это будет шумом для наших метрик  
Проанализируем и удалим

## Разведочный анализ

In [ ]:
df.to_csv(f"{x}_cleared_divvy.csv", index=False)
day.to_csv(f"{x}_bigger_one_day.csv", index=False)
fix_df.to_csv(f"{x}_fixing_data.csv", index=False)
na_end_coord.to_csv(f"{x}_na_end_coord.csv", index=False)

In [ ]:
sns.histplot(df["month"], bins=12)

In [ ]:
plt.rcParams.update({"font.size": 12})
fig, ax = plt.subplots(1, 2, figsize=(15,5))
sns.histplot(df[df["member_casual"] == "member"]["month"], bins=12, ax=ax[0])
sns.histplot(df[df["member_casual"] == "casual"]["month"], bins=12, ax=ax[1])
ax[0].set_xlabel("members")
ax[1].set_xlabel("casual")

In [ ]:
sns.barplot(df, x="member_casual")